In [1]:
# 라이브러리 설치
!pip install pycaret==3.0.0
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
# 데이터 업로드
from google.colab import files
uploaded = files.upload()

Saving ObesityDataSet.csv to ObesityDataSet.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [2]:
# 데이터 가져오기
df_test = pd.read_csv('test.csv', encoding='cp949')
df_train = pd.read_csv('train.csv', encoding='cp949')
df_original= pd.read_csv('ObesityDataSet.csv', encoding='cp949')

print(df_test.shape, df_train.shape, df_original.shape)
# >> (13840, 17) (20758, 18) (2111, 17)

(13840, 17) (20758, 18) (2111, 17)


In [ ]:
# 데이터 확인하기
display(df_test.head()) # pandas 함수
display(df_train.head())
display(df_original.head())

# >> test의 경우 NObeyesdad 컬럼이 없고 original의 경우 id 컬럼이 없음

In [ ]:
# 결측치 확인
print(df_test.isna().sum())
print(df_train.isna().sum())
print(df_original.isna().sum(), '\n\n\n')

# >> 결측치가 없으므로 drop하거나 fill, dropna 따위를 할 필요가 없음

# 중복값 확인
df_test.drop_duplicates()
df_train.drop_duplicates()
df_original.drop_duplicates()

print(df_test.shape, df_train.shape, df_original.shape)

# 기존: (13840, 17) (20758, 18) (2111, 17)
# >> (13840, 17) (20758, 18) (2111, 17), 차이가 없으므로 중복값도 없음

In [ ]:
# train 데이터 생성 및 불필요한 id 컬럼 제거
train =  pd.concat([df_train, df_original]).drop(['id'], axis=1).drop_duplicates()
test = df_test.drop('id', axis = 1)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

# 피쳐 엔지니어링: 새로운 변수 만들기 - BMI
def df_bmi(df):
    df['BMI'] = df['Weight']/(df['Height']**2)

df_bmi(train)
df_bmi(test)

print(train.shape, test.shape, '\n\n')
# >> (22845, 18) (13840, 16),  train에서 24개의 중복값이 존재했었음

print(train.info()) # 데이터 타입 확인

In [6]:
# 외부에서 EDA 진행, 분석 후 피쳐 엔지니어링 더 진행하기
train.to_csv('train_concat_original.csv', index=False)

In [7]:
# 종속변수와 독립변수 분리
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(train['NObeyesdad'])
print(encoder.classes_)
# >> ['Insufficient_Weight' 'Normal_Weight' 'Obesity_Type_I' 'Obesity_Type_II' 'Obesity_Type_III' 'Overweight_Level_I' 'Overweight_Level_II']
X = train.drop(['NObeyesdad'], axis = 1)
y = y_encoded

['Insufficient_Weight' 'Normal_Weight' 'Obesity_Type_I' 'Obesity_Type_II'
 'Obesity_Type_III' 'Overweight_Level_I' 'Overweight_Level_II']


In [8]:
# 데이터 전처리 (1)
"""
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

cat_cols = X.select_dtypes(include=['object']) # 숫자 컬럼 분리
num_cols = X.select_dtypes(include=['float64']) # 문자 컬럼 분리

preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ], remainder = 'passthrough'
)
"""

# 데이터 전처리 (2)
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.compose import ColumnTransformer

cat_cols = X.select_dtypes(include=['object']).columns # 숫자 컬럼 분리
num_cols = X.select_dtypes(include=['float64']).columns # 문자 컬럼 분리

num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(
        handle_unknown='use_encoded_value',
        unknown_value = -1
    ))
])
# display(OrdinalEncoder().fit(cat_cols).categories_)

preprocessor = ColumnTransformer(
    transformers = [
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, cat_cols)
    ], remainder = 'passthrough'
)

# 데이터 전처리 (3)
"""
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
"""

'\nfrom sklearn.preprocessing import MinMaxScaler\nfrom sklearn.preprocessing import MaxAbsScaler\nfrom sklearn.preprocessing import RobustScaler\nfrom sklearn.preprocessing import Normalizer\n'

In [9]:
# 데이터셋 분리, 샘플링 (1) - random
"""
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.3,
    random_state = 42
)
"""

# 데이터셋 분리, 샘플링 (2) - startify
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    stratify = y,
    test_size = 0.3,
    random_state = 42
)

# 데이터셋 분리, 샘플링(3)
"""
SMOTE
"""

'\nSMOTE\n'

In [10]:
# pycaret을 활용하여 모델 선정

from pycaret.classification import *
exp_clf = setup(data = X_train, target = y_train, session_id=42)

,Description,Value
0,Session id,42
1,Target,target
2,Target type,Multiclass
3,Original data shape,"(15991, 18)"
4,Transformed data shape,"(15991, 28)"
5,Transformed train set shape,"(11193, 28)"
6,Transformed test set shape,"(4798, 28)"
7,Ordinal features,5
8,Numeric features,9
9,Categorical features,8


In [11]:
best_model = compare_models()
best_model

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9058,0.9903,0.9058,0.9060,0.9056,0.8896,0.8897,6.7920
xgboost,Extreme Gradient Boosting,0.9049,0.9903,0.9049,0.9051,0.9047,0.8886,0.8887,2.2860
rf,Random Forest Classifier,0.9020,0.9891,0.9020,0.9024,0.9018,0.8851,0.8852,2.3510
gbc,Gradient Boosting Classifier,0.8999,0.9892,0.8999,0.9002,0.8996,0.8827,0.8828,16.5250
et,Extra Trees Classifier,0.8897,0.9869,0.8897,0.8897,0.8892,0.8706,0.8708,2.2340
knn,K Neighbors Classifier,0.8785,0.9675,0.8785,0.8782,0.8780,0.8576,0.8577,0.6310
dt,Decision Tree Classifier,0.8415,0.9086,0.8415,0.8424,0.8415,0.8142,0.8144,0.6060
lda,Linear Discriminant Analysis,0.8329,0.9770,0.8329,0.8333,0.8313,0.8043,0.8048,0.4710
lr,Logistic Regression,0.8328,0.9755,0.8328,0.8295,0.8303,0.8039,0.8041,4.3170
nb,Naive Bayes,0.6519,0.9392,0.6519,0.6599,0.6213,0.5914,0.6008,0.7080


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [12]:
lgbm = create_model('lightgbm') # 가장 좋은 거 선택
tuned_lg = tune_model(lgbm) # 하이퍼 파라미터 튜닝
tuned_lg

tuned_lg.get_params() # 튜닝 결과 보기

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9152,0.9930,0.9152,0.9151,0.9149,0.9006,0.9007
1,0.8955,0.9885,0.8955,0.8953,0.8947,0.8775,0.8777
2,0.9036,0.9905,0.9036,0.9042,0.9030,0.8870,0.8872
3,0.9187,0.9913,0.9187,0.9198,0.9188,0.9047,0.9048
4,0.9053,0.9897,0.9053,0.9043,0.9045,0.8889,0.8890
5,0.9160,0.9908,0.9160,0.9157,0.9158,0.9015,0.9016
6,0.8990,0.9896,0.8990,0.8999,0.8989,0.8816,0.8818
7,0.9035,0.9891,0.9035,0.9037,0.9035,0.8869,0.8869
8,0.8981,0.9910,0.8981,0.8996,0.8984,0.8806,0.8807


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9179,0.9925,0.9179,0.9175,0.9176,0.9037,0.9037
1,0.8991,0.9900,0.8991,0.8988,0.8983,0.8817,0.8819
2,0.9134,0.9907,0.9134,0.9127,0.9126,0.8985,0.8986
3,0.9178,0.9918,0.9178,0.9182,0.9177,0.9036,0.9037
4,0.8910,0.9893,0.8910,0.8904,0.8905,0.8722,0.8722
5,0.9142,0.9911,0.9142,0.9138,0.9139,0.8994,0.8995
6,0.9044,0.9898,0.9044,0.9043,0.9041,0.8879,0.8880
7,0.8990,0.9893,0.8990,0.8999,0.8992,0.8817,0.8817
8,0.9026,0.9917,0.9026,0.9040,0.9029,0.8858,0.8860


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.2,
 'max_depth': -1,
 'min_child_samples': 6,
 'min_child_weight': 0.001,
 'min_split_gain': 0.6,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 30,
 'objective': None,
 'random_state': 42,
 'reg_alpha': 0.001,
 'reg_lambda': 5,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'feature_fraction': 0.8,
 'bagging_freq': 3,
 'bagging_fraction': 0.8}

In [13]:
print(tuned_lg.get_params())

{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.2, 'max_depth': -1, 'min_child_samples': 6, 'min_child_weight': 0.001, 'min_split_gain': 0.6, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 30, 'objective': None, 'random_state': 42, 'reg_alpha': 0.001, 'reg_lambda': 5, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'feature_fraction': 0.8, 'bagging_freq': 3, 'bagging_fraction': 0.8}


In [14]:
# 모델링
from lightgbm import LGBMClassifier

param_grid = {'boosting_type': 'gbdt',
              'class_weight': None,
              'colsample_bytree': 1.0,
              'importance_type': 'split',
              'learning_rate': 0.2,
              'max_depth': -1,
              'min_child_samples': 6,
              'min_child_weight': 0.001,
              'min_split_gain': 0.6,
              'n_estimators': 100,
              'n_jobs': -1,
              'num_leaves': 30,
              'objective': None,
              'random_state': 42,
              'reg_alpha': 0.001,
              'reg_lambda': 5,
              'subsample': 1.0,
              'subsample_for_bin': 200000,
              'subsample_freq': 0,
              'feature_fraction': 0.8,
              'bagging_freq': 3,
              'bagging_fraction': 0.8}

lgbm = LGBMClassifier(**param_grid)

pipeline = Pipeline(steps=[ # 파이프 라인 구축
    ('preprocessor', preprocessor),
    ('clf', lgbm)
])

# 모델 트레이닝
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003576 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2280
[LightGBM] [Info] Number of data points in the train set: 15991, number of used features: 17
[LightGBM] [I

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  Index(['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'BMI'], dtype...
                                boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, feature_fraction=0.8,
                                importance_type='split', learning_rate=0.2,
                                max_depth=-1, min_child_samples=6,
                                min_child_weight=0.001, min_split_gain=0.6,
                                n_estimators=100, n_jobs=-1, num_leaves=30,
                                objective=None, random_state=42,
                                reg_alpha=0.001, reg_lambda=5, subsample=1.0,
                                subsample_for_bin=200000, subsample_freq=0))],
         verbose=False)

In [15]:
# 모델 예측 및 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

recall = recall_score(y_test, y_pred, average='macro')
print(recall)

f1 = f1_score(y_test, y_pred, average='macro')
print(f1)

[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
0.9101254741756638
0.9009668802801672
0.9009489765964479
